In [2]:
from sklearn.model_selection import train_test_split
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense
import numpy as np

In [3]:
y = pd.read_csv(r"C:\Users\anant\Documents\GitHub\Taxonomic_classifier\Data\Taxonomic_data.csv", usecols=['Species'])
x = pd.read_csv(r"C:\Users\anant\Documents\GitHub\Taxonomic_classifier\Data\k(7)mer_data.csv").drop("Unnamed: 0", axis=1)

In [4]:
from sklearn.decomposition import PCA
pca = PCA(n_components=0.95)
xx = pca.fit_transform(x)
pca.inverse_transform(xx)
pca.explained_variance_ratio_.sum()
print(f"{x.shape[1]} columns originally present, \n{pca.n_components_} account for 95 percent variance ")
print(f"corresponds to {100*pca.n_components_/x.shape[1]}% of total")

16384 columns originally present, 
840 account for 95 percent variance 
corresponds to 5.126953125% of total


In [ ]:
train_x, test_x, train_y, test_y = train_test_split(x, y, test_size=0.2, random_state=7)

In [ ]:
train_y.describe()

,Species
count,10601
unique,2351
top,Escherichia coli
freq,712


In [5]:
train_x.shape

(10601, 16384)

In [6]:
from sklearn import preprocessing
lb = preprocessing.LabelBinarizer()
lb.fit(y)
test_y = lb.transform(test_y)
train_y = lb.transform(train_y)


In [7]:
model = Sequential()
model.add(Dense(20, input_shape =(None, train_x.shape[1]) , activation="relu"))
model.add(Dense(len(y.Species.unique()),activation='softmax'))

In [8]:
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, None, 20)          327700    
                                                                 
 dense_1 (Dense)             (None, None, 2621)        55041     
                                                                 
Total params: 382,741
Trainable params: 382,741
Non-trainable params: 0
_________________________________________________________________


In [9]:
model.fit(train_x, train_y, epochs = 25)

Epoch 1/25
332/332 [==============================] - 3s 6ms/step - loss: 4.7441 - accuracy: 0.3447
Epoch 2/25
332/332 [==============================] - 2s 6ms/step - loss: 2.7617 - accuracy: 0.5405
Epoch 3/25
332/332 [==============================] - 2s 6ms/step - loss: 1.8461 - accuracy: 0.6568
Epoch 4/25
332/332 [==============================] - 2s 6ms/step - loss: 1.2958 - accuracy: 0.7232
Epoch 5/25
332/332 [==============================] - 2s 6ms/step - loss: 0.9365 - accuracy: 0.7679
Epoch 6/25
332/332 [==============================] - 2s 6ms/step - loss: 0.7188 - accuracy: 0.8078
Epoch 7/25
332/332 [==============================] - 2s 6ms/step - loss: 0.5749 - accuracy: 0.8348
Epoch 8/25
332/332 [==============================] - 2s 7ms/step - loss: 0.4844 - accuracy: 0.8542
Epoch 9/25
332/332 [==============================] - 2s 7ms/step - loss: 0.4233 - accuracy: 0.8688
Epoch 10/25
332/332 [==============================] - 2s 7ms/step - loss: 0.3797 - accuracy: 0.8773

In [10]:
model.evaluate(test_x, test_y)

83/83 [==============================] - 1s 7ms/step - loss: 2.8426 - accuracy: 0.8257


[2.8425960540771484, 0.8257261514663696]

In [11]:
from sklearn import metrics
f1 = metrics.f1_score( lb.inverse_transform(test_y), lb.inverse_transform(model.predict(test_x)), average='weighted')
f1

0.809366640546745

In [14]:
i = 0
mismatch = 0
npredictions = 0
predictions = []
mismatch_species = []
mismatch_species_pred = []
y_tested = []
for y in test_y:
    a = model.predict(test_x[i:i+1]) 
    if a.max()>0.8:
        sp = lb.inverse_transform(np.array([y]))
        p_sp = lb.inverse_transform(a)
        if p_sp!=sp :
            mismatch_species.append(sp)
            mismatch_species_pred.append(p_sp)
            mismatch += 1
        npredictions += 1
        predictions.append(p_sp)
        y_tested.append(sp)
    i  += 1
print(mismatch)

129


In [ ]:
from sklearn import metrics
f1 = metrics.f1_score(y_predicted, predictions, average = 'weighted')
f1

In [15]:
mismatch

129

In [16]:
mismatch_species_pred

[array(['Micromonospora sp. L5'], dtype='<U50'),
 array(['Escherichia coli'], dtype='<U50'),
 array(['Enterobacter aerogenes'], dtype='<U50'),
 array(['Pyrolobus fumarii'], dtype='<U50'),
 array(['Micromonospora sp. L5'], dtype='<U50'),
 array(['Pseudomonas fuscovaginae'], dtype='<U50'),
 array(['Bacillus mycoides'], dtype='<U50'),
 array(['Borrelia burgdorferi'], dtype='<U50'),
 array(['Eubacteriaceae bacterium ACC19a'], dtype='<U50'),
 array(['Burkholderia sp. YI23'], dtype='<U50'),
 array(['Escherichia coli'], dtype='<U50'),
 array(['Haemophilus paraphrohaemolyticus'], dtype='<U50'),
 array(['Burkholderia xenovorans'], dtype='<U50'),
 array(['Streptomyces sp. C'], dtype='<U50'),
 array(['Bacteroides fragilis'], dtype='<U50'),
 array(['Lactobacillus johnsonii'], dtype='<U50'),
 array(['Achromobacter xylosoxidans'], dtype='<U50'),
 array(['Rickettsia rickettsii'], dtype='<U50'),
 array(['Clostridium botulinum'], dtype='<U50'),
 array(['Bradyrhizobium sp. S23321'], dtype='<U50'),
 arra

In [ ]:
mismatch_species

[array(['Candidatus Regiella insecticola'], dtype='<U50'),
 array(['Escherichia albertii'], dtype='<U50'),
 array(['Bacillus anthracis'], dtype='<U50'),
 array(['Enterococcus saccharolyticus'], dtype='<U50'),
 array(['Micromonospora aurantiaca'], dtype='<U50'),
 array(['Erysipelotrichaceae bacterium 21_3'], dtype='<U50'),
 array(['Gemella haemolysans'], dtype='<U50'),
 array(['Borrelia sp. SV1'], dtype='<U50'),
 array(['Terriglobus saanensis'], dtype='<U50'),
 array(['Burkholderia sp. SJ98'], dtype='<U50'),
 array(['Escherichia sp. TW09308'], dtype='<U50'),
 array(['Fusobacterium nucleatum'], dtype='<U50'),
 array(['Francisella novicida'], dtype='<U50'),
 array(['Burkholderia oklahomensis'], dtype='<U50'),
 array(['Haemophilus parahaemolyticus'], dtype='<U50'),
 array(['Bacteroides sp. 2_1_56FAA'], dtype='<U50'),
 array(['Lactobacillus hominis'], dtype='<U50'),
 array(['Aciduliprofundum sp. MAR08-339'], dtype='<U50'),
 array(['Rickettsia philipii'], dtype='<U50'),
 array(['Pseudomonas 

In [ ]:
i
# 462

2651

In [18]:
predictions

2170

In [20]:
model.save("k7-net")

INFO:tensorflow:Assets written to: k7-net\assets
